In [6]:
import pandas as pd
import os
import nest_asyncio
nest_asyncio.apply()
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

SEED = 1337
tf.random.set_seed(SEED)
# Get the current working directory
current_dir = os.getcwd()

# Change the working directory to the parent directory of the current directory
os.chdir(os.path.join(current_dir, '..'))
# Enter path to the data_fusion_guest_lecture file
image_folder_path = 'data/seedling_labels_with_features.csv'

# Loads labels
df = pd.read_csv(image_folder_path)

df = df.drop(columns=['Pos','average_expert'])
df = df.iloc[:, -8:]

FileNotFoundError: [Errno 2] No such file or directory: 'data\\seedling_labels_with_features.csv'

In [9]:
# group the dataframe by the 'Rfid' column
groups = df.groupby('Rfid')

# create separate dataframes for each group and drop the 'Rfid' column
dfs = []
i = 1
for _, group_data in groups:
    df_name = f"df_{i}"
    globals()[df_name] = group_data.drop('Rfid', axis=1)
    dfs.append(globals()[df_name] )
    i += 1


In [10]:
df['expert_binary'].value_counts()

1    673
0    321
Name: expert_binary, dtype: int64

In [13]:
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

SEED = 1337
tf.random.set_seed(SEED)

In [31]:
BATCH_SIZE = 128


def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
    dataset = dataframe

    # Class balancing
    pos_df = dataset[dataset['expert_binary'] == 1]
    neg_df = dataset[dataset['expert_binary'] == 0]
    
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('expert_binary')
    
    # Dataset creation
    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(2048, seed=SEED)
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

train_data, val_data = [], []
for client_data in dfs:
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=SEED)

    # Scaling (Standardization actually hurts performance) 
    scaler = MinMaxScaler() 
    train_features = scaler.fit_transform(train_df.drop(['expert_binary'], axis=1))
    val_features = scaler.transform(val_df.drop(['expert_binary'], axis=1))

    train_df[train_df.columns.difference(['expert_binary'])] = train_features
    val_df[val_df.columns.difference(['expert_binary'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE))
    val_data.append(make_tf_dataset(val_df, batch_size=1))

def input_spec():
    return (
        tf.TensorSpec([None, 6], tf.float64),           #you need to change the expected input shape based on the number of features we have
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(6,)),   #you need to change the input shape based on the number of features we have
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[BinaryAccuracy(), Precision(), Recall()])

training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

#print(training_process.initialize.type_signature.formatted_representation())

train_state = training_process.initialize()

NUM_ROUNDS = 110
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, train_data)
  train_state = result.state
  train_metrics = result.metrics['client_work']['train']
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('binary_accuracy', 0.67227834), ('precision', 0.6719101), ('recall', 1.0), ('loss', 0.64619756), ('num_examples', 891), ('num_batches', 8)])
round  3, metrics=OrderedDict([('binary_accuracy', 0.671156), ('precision', 0.671156), ('recall', 1.0), ('loss', 0.635824), ('num_examples', 891), ('num_batches', 8)])
round  4, metrics=OrderedDict([('binary_accuracy', 0.671156), ('precision', 0.671156), ('recall', 1.0), ('loss', 0.62593406), ('num_examples', 891), ('num_batches', 8)])
round  5, metrics=OrderedDict([('binary_accuracy', 0.671156), ('precision', 0.671156), ('recall', 1.0), ('loss', 0.61646146), ('num_examples', 891), ('num_batches', 8)])
round  6, metrics=OrderedDict([('binary_accuracy', 0.671156), ('precision', 0.671156), ('recall', 1.0), ('loss', 0.6073237), ('num_examples', 891), ('num_batches', 8)])
round  7, metrics=OrderedDict([('binary_accuracy', 0.671156), ('precision', 0.671156), ('recall', 1.0), ('loss', 0.5984809), ('num_examples', 891), ('

In [29]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [30]:
federated_metrics = evaluator(training_process.get_model_weights(train_state), val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9223301),
                           ('precision', 0.9350649),
                           ('recall', 0.96),
                           ('loss', 0.16753183),
                           ('num_examples', 103),
                           ('num_batches', 103)]))])

In [21]:
train_data = train_data[0].concatenate(train_data[1])
val_data = val_data[0].concatenate(val_data[1])


In [22]:

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(6,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[BinaryAccuracy(), Precision(), Recall()],
    )
    
    return model

model = model_fn()
history = model.fit(train_data, epochs=EPOCHS)

test_scores = model.evaluate(val_data)
single_metrics = {
    'loss': test_scores[0],
    'binary_accuracy': test_scores[1],
    'precision': test_scores[2],
    'recall': test_scores[3]
}
single_metrics

Epoch 1/100
4/4 [==============================] - 1s 3ms/step - loss: 0.7139 - binary_accuracy: 0.3333 - precision: 0.5854 - recall: 0.1529    
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 0.6638 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.6207 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 0.5834 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 0.5536 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 0.5292 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 0.5116 - binary_accuracy: 0.6978 - precision: 0.6978 - recall: 1.0000
Epoch 8/1

{'loss': 0.11324683576822281,
 'binary_accuracy': 0.9615384340286255,
 'precision': 1.0,
 'recall': 0.9473684430122375}

In [24]:
print(f"---Single model metrics---\n{single_metrics}\n")
print(f"---Federated model metrics---\n{dict(federated_metrics['eval'])}")

---Single model metrics---
{'loss': 0.11324683576822281, 'binary_accuracy': 0.9615384340286255, 'precision': 1.0, 'recall': 0.9473684430122375}

---Federated model metrics---
{'binary_accuracy': 0.9223301, 'precision': 0.9350649, 'recall': 0.96, 'loss': 0.22869204, 'num_examples': 103, 'num_batches': 103}
